In [1]:
import glob
import gzip
import os
import shutil
import sys
import time
import warnings
import zipfile
from datetime import date

import cdsapi
import h5py
import numpy
import numpy as np
import pandas as pd
import urllib3
import xarray

warnings.filterwarnings("ignore")
import pycountry

sys.path.append(os.getcwd() + "/../cds-backend/code/")
import copy
import glob
import json

import cds_eua4 as eua
import numba
import pandas
import psutil
import requests
from numba import njit


def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]


def datetime_to_seconds(dates, ref="1900-01-01T00:00:00"):
    """from datetime64 to seconds since 1900-01-01 00:00:00"""
    return ((dates - np.datetime64(ref)) / np.timedelta64(1, "s")).astype(np.int64)


def seconds_to_datetime(seconds, ref="1900-01-01"):
    """from seconds to datetime64"""
    seconds = np.asarray(seconds)
    return pd.to_datetime(seconds, unit="s", origin=ref)


import matplotlib
import matplotlib.pylab as plt
import matplotlib.pyplot as maplt

matplotlib.rcParams.update({"font.size": 20})
matplotlib.rcParams["figure.figsize"] = (20, 10)
font = {
    "family": "normal",
    # 'weight' : 'bold',
    "size": 22,
}
matplotlib.rc("font", **font)

In [3]:
stations = glob.glob("/mnt/users/scratch/leo/scratch/converted_v11/long/*.nc")
stations[:10]

['/mnt/users/scratch/leo/scratch/converted_v11/long/0-20999-0-01139_era5_2_harvested_era5.conv._01139.gz.nc_CEUAS_merged_v1.nc',
 '/mnt/users/scratch/leo/scratch/converted_v11/long/0-20999-0-01653_era5_2_harvested_era5.conv._01653.gz.nc_CEUAS_merged_v1.nc',
 '/mnt/users/scratch/leo/scratch/converted_v11/long/0-20999-0-3:0L0_era5_2_harvested_era5.conv._3:0L0.gz.nc_CEUAS_merged_v1.nc',
 '/mnt/users/scratch/leo/scratch/converted_v11/long/0-20999-0-00345_era5_2_harvested_era5.conv._00345.gz.nc_CEUAS_merged_v1.nc',
 '/mnt/users/scratch/leo/scratch/converted_v11/long/0-20999-0-3:4YB_era5_2_harvested_era5.conv._3:4YB.gz.nc_CEUAS_merged_v1.nc',
 '/mnt/users/scratch/leo/scratch/converted_v11/long/0-20999-0-00432_era5_2_harvested_era5.conv._00432.gz.nc_CEUAS_merged_v1.nc',
 '/mnt/users/scratch/leo/scratch/converted_v11/long/0-124-0-73033_CEUAS_merged_v1.nc',
 '/mnt/users/scratch/leo/scratch/converted_v11/long/0-20999-0-01162_era5_2_harvested_era5.conv._01162.gz.nc_CEUAS_merged_v1.nc',
 '/mnt/use

In [63]:
bad_stats = []
for stat in stations[:20]:
    print(stat)
    with eua.CDMDataset(stat) as file:
        df = file.to_dataframe(
            groups=["observations_table"],
            variables=[
                "date_time",
                "z_coordinate",
                "z_coordinate_type",
                "z_coordinate_method",
                "observed_variable",
                "observation_value",
                "conversion_flag",
            ],
        )
    df_ta = df[df.observed_variable == 126]
    df_dpd = df[df.observed_variable == 34]
    df_dp = df[df.observed_variable == 137]
    df_rh = df[df.observed_variable == 138]
    df_rh = df_rh.rename(columns={"observation_value": "rh"})
    df_dp = df_dp.rename(columns={"observation_value": "dp"})
    df_ta = df_ta.rename(columns={"observation_value": "ta"})

    print("df_ta", len(df_ta))
    print("df_dpd", len(df_dpd))
    print("df_dp", len(df_dp))
    print("df_rh", len(df_rh))
    df_ta_dp = pandas.merge(
        df_dp[["date_time", "z_coordinate", "dp", "conversion_flag"]],
        df_ta[["date_time", "z_coordinate", "ta"]],
        on=["date_time", "z_coordinate"],
        how="left",
    )
    df_hum = pandas.merge(
        df_ta_dp,
        df_rh[["date_time", "z_coordinate", "rh"]],
        on=["date_time", "z_coordinate"],
        how="left",
    )
    df_check = df_hum[df_hum.dp > df_hum.ta]
    if len(df_check) > 0:
        display(df_check)
    print()
    print()

/mnt/users/scratch/leo/scratch/converted_v11/long/0-20999-0-01139_era5_2_harvested_era5.conv._01139.gz.nc_CEUAS_merged_v1.nc
df_ta 156
df_dpd 95
df_dp 101
df_rh 97


,date_time,z_coordinate,dp,conversion_flag,ta,rh
17,1968-03-23 00:00:00,99100.0,275.299988,-2147483648,273.399994,NaN
54,1970-12-18 23:00:00,99100.0,275.000000,-2147483648,271.500000,NaN
86,1972-02-05 00:00:00,102100.0,273.600006,-2147483648,269.899994,NaN
90,1972-07-02 00:00:00,23800.0,288.200012,-2147483648,187.899994,NaN




/mnt/users/scratch/leo/scratch/converted_v11/long/0-20999-0-01653_era5_2_harvested_era5.conv._01653.gz.nc_CEUAS_merged_v1.nc
df_ta 21
df_dpd 5
df_dp 5
df_rh 5


/mnt/users/scratch/leo/scratch/converted_v11/long/0-20999-0-3:0L0_era5_2_harvested_era5.conv._3:0L0.gz.nc_CEUAS_merged_v1.nc
df_ta 563
df_dpd 460
df_dp 460
df_rh 504


/mnt/users/scratch/leo/scratch/converted_v11/long/0-20999-0-00345_era5_2_harvested_era5.conv._00345.gz.nc_CEUAS_merged_v1.nc
df_ta 1246
df_dpd 612
df_dp 632
df_rh 631


,date_time,z_coordinate,dp,conversion_flag,ta,rh
476,1970-01-12 23:00:00,22900.0,299.200012,-2147483648,243.100006,NaN




/mnt/users/scratch/leo/scratch/converted_v11/long/0-20999-0-3:4YB_era5_2_harvested_era5.conv._3:4YB.gz.nc_CEUAS_merged_v1.nc
df_ta 144213
df_dpd 66507
df_dp 66507
df_rh 72268


/mnt/users/scratch/leo/scratch/converted_v11/long/0-20999-0-00432_era5_2_harvested_era5.conv._00432.gz.nc_CEUAS_merged_v1.nc
df_ta 687
df_dpd 351
df_dp 359
df_rh 356


...

,date_time,z_coordinate,dp,conversion_flag,ta,rh
43,1968-03-09 01:00:00,99400.0,276.100006,-2147483648,270.899994,NaN
44,1968-03-15 00:00:00,102000.0,277.899994,-2147483648,245.100006,NaN




/mnt/users/scratch/leo/scratch/converted_v11/long/0-20999-0-3:GHZM_era5_2_harvested_era5.conv._3:GHZM.gz.nc_CEUAS_merged_v1.nc
df_ta 1
df_dpd 1
df_dp 1
df_rh 1


/mnt/users/scratch/leo/scratch/converted_v11/long/0-12-20000-60580_CEUAS_merged_v1.nc
df_ta 37140
df_dpd 21254
df_dp 22552
df_rh 26556


,date_time,z_coordinate,dp,conversion_flag,ta,rh
4795,1966-12-14 11:00:00,20000.0,223.199997,-2147483648,219.199997,NaN
5129,1967-01-14 11:00:00,14000.0,275.200012,-2147483648,219.500000,NaN
8987,1968-01-18 11:00:00,100900.0,287.200012,-2147483648,284.600006,NaN
9032,1968-01-23 11:00:00,101000.0,281.200012,-2147483648,260.700012,NaN
9051,1968-01-25 11:00:00,90000.0,287.200012,-2147483648,284.799988,NaN
...,...,...,...,...,...,...
17915,1970-05-08 11:00:00,20800.0,303.200012,-2147483648,299.200012,NaN
17991,1970-05-15 11:00:00,99300.0,295.200012,-2147483648,294.399994,1.05014
19793,1970-11-07 11:00:00,100100.0,297.200012,-2147483648,294.399994,NaN
19805,1970-11-08 12:00:00,100300.0,293.200012,-2147483648,291.600006,NaN




/mnt/users/scratch/leo/scratch/converted_v11/long/0-20999-0-3:0K3K_era5_2_harvested_era5.conv._3:0K3K.gz.nc_CEUAS_merged_v1.nc
df_ta 163
df_dpd 0
df_dp 0
df_rh 0


/mnt/users/scratch/leo/scratch/converted_v11/long/0-20999-0-00731_era5_2_harvested_era5.conv._00731.gz.nc_CEUAS_merged_v1.nc
df_ta 472
df_dpd 295
df_dp 317
df_rh 312


,date_time,z_coordinate,dp,conversion_flag,ta,rh
101,1968-02-05 12:00:00,102200.0,275.200012,-2147483648,271.899994,NaN
143,1969-01-02 00:00:00,99000.0,276.200012,-2147483648,270.899994,NaN
197,1969-10-05 12:00:00,20800.0,310.200012,-2147483648,298.200012,NaN
223,1970-08-14 00:00:00,28000.0,305.200012,-2147483648,248.699997,NaN
243,1970-12-28 10:00:00,98600.0,274.100006,-2147483648,272.500000,NaN
